# Transformer

In [1]:
import torch
import torch.nn as nn
import math

/home/dean/miniconda3/envs/vln/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Positional Encoding

$$
\text{PE}(\text{pos}, 2i) = sin \left( \frac{\text{pos}}{10000^{2i/d}} \right) \\
\text{PE}(\text{pos}, 2i+1) = cos \left( \frac{\text{pos}}{10000^{2(i+1)/d}} \right)
$$


In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len=80):
        self.d_model = d_model
        
        # create a const matrix PE accodring to pos and i
        pe = torch.zeros(max_sex_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i+1) / d_model))))
                                          
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)  
    
    def forward(self, x):
        # scale the x, make its value bigger
        x = x * math.sqrt(self.d_model)
        seq_len = x.size(1)
        x = x + Variable(self.pe[:, :seq_len], requires_grad=False).cuda()
        return x

## Multi-head Attention

$$
Z = \text{Attention}(Q, K ,V) = \text{Softmax}(\frac{QK^T}{\sqrt{d}})V
$$

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def attention(q, k, v, d_k, mask=None, dropout=None):
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
        
        if mask is not None:
            mask = mask.unsqueeze()
            scores = scores.masked_fill(mask == 0, -1e9)
        
        scores = F.softmax(scores, dim = -1)
        
        if dropout is not None:
            scores = dropout(scores)
        
        output = torch.matmul(scores, v)
        return output
            
    
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
        
        output = self.output(concat)
        
        return output

### Mask
```python
# create mask for the input
batch = next(iter(train_iter))
input_seq = batch.English.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']
input_msk = (input_seq != input_pad).unsqueeze(1)

# create mask for terget seq
target_seq = batch.French.transpose(0,1)
target_pad = FR_TEXT.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)
size = target_seq.size(1) # get seq_len for matrix
nopeak_mask = np.triu(np.ones(1, size, size), k=1).astype('uint8')
nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
target_msk = target_msk & nopeak_mask
```

## Feed Forward Network

$$
\text{FFN}(x) = \text{Relu}(xW_1+b_1)W_2+b_2
$$

In [4]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

## Norm Layer

$$
\text{LN}(x) = \alpha \cdot \frac{x-\mu} {\sigma} + b
$$

In [5]:
class NormLayer(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        
        self.size = d_model
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
        
    def forward(self, x):
        norm = (x - x.mean(dim=-1, keepdim=True)) / (x.std(dim=-1, keepdim=True) + self.eps)
        norm = self.alpha *  tmp + self.bias
        return norm

## Encoder

In [6]:
def get_clones(module, N):
    
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm1 = NormLayer(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm2 = NormLayer(d_model)
        self.ffn = FeedForward(d_model, dropout=dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x_norm = self.norm1(x)
        x = x + self.dropout1(self.attn(x_norm, x_norm, x_norm, mask))
        x_norm = self.norm2(x)
        x = x + self.dropout2(self.ffn(x_norm))
        return x

In [8]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout, max_seq_len=80):
        super().__init__()
        self.N = N
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = PositionalEncoding(d_model, max_seq_len)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = NormLayer(d_model)
    
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)